In [33]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import pandas as pd

In [39]:
from fiona.drvsupport import supported_drivers
supported_drivers['LIBKML'] = 'rw'

stations_fp = './stations.kml'
layers = gpd.list_layers(stations_fp)['name']
transit_lines = layers[0:-1] # drop 'Game Area'
transit_lines

0               1Line
1    RapidRide C Line
2    RapidRide E Line
3    RapidRide D Line
4    RapidRide F Line
5    RapidRide G Line
6    RapidRide H Line
7    SoundTransit 522
8           Streetcar
Name: name, dtype: object

In [91]:
line_gdfs = []

for line in transit_lines:
    gdf = gpd.read_file(stations_fp, columns = ['Name', 'Latitude', 'Longitude'], layer = line)
    gdf['Line'] = line
    line_gdfs.append(gdf)

stations = pd.concat(line_gdfs)
stations.rename(columns={'geometry': 'Center'}, inplace=True)
cols = ['Line', 'Name', 'Latitude', 'Longitude', 'Center']
stations = stations[cols]
stations

,Line,Name,Latitude,Longitude,Center
0,1Line,Lynnwood City Center,47.815637,-122.294779,POINT Z (-122.29478 47.81564 0)
1,1Line,Mountlake Terrace,47.785057,-122.314788,POINT Z (-122.31479 47.78506 0)
2,1Line,Shoreline North/185th,47.764067,-122.322889,POINT Z (-122.32289 47.76407 0)
3,1Line,Shoreline South/148th,47.736095,-122.325227,POINT Z (-122.32523 47.7361 0)
4,1Line,Northgate,47.703028,-122.32829,POINT Z (-122.32829 47.70303 0)
...,...,...,...,...,...
27,Streetcar,Terry & Mercer,47.6234169,-122.337044,POINT Z (-122.33704 47.62342 0)
28,Streetcar,Fairview & Campus Drive,47.6276665,-122.332367,POINT Z (-122.33237 47.62767 0)
29,Streetcar,Lake Union Park,47.6258316,-122.336372,POINT Z (-122.33637 47.62583 0)
30,Streetcar,Lake Union Park,47.6259689,-122.336533,POINT Z (-122.33653 47.62597 0)


In [103]:
radius_mi = 0.3
radius_m = radius_mi * 5280 * 0.3048
hiding_areas = stations.set_geometry('Center').to_crs(3857).buffer(radius_m)
stations['Buffer'] = hiding_areas
stations.set_geometry('Buffer').explore(column='Line')